# Configuration et Installation TweetyProject

**Navigation**: Index | [Suivant: Tweety-2-Basic-Logics](Tweety-2-Basic-Logics.ipynb)

---

Ce notebook configure l'environnement pour la série de notebooks TweetyProject qui explore la bibliothèque Java [TweetyProject](https://tweetyproject.org/) pour l'intelligence artificielle symbolique.

**IMPORTANT:** Ce notebook **Tweety-1-Setup** configure uniquement l'environnement. Les exemples de logiques et d'argumentation se trouvent dans les notebooks suivants de la série.

## Objectifs pédagogiques

1. Comprendre l'architecture TweetyProject (JARs Java + JPype)
2. Configurer l'environnement Python/Java pour Tweety
3. Vérifier le bon fonctionnement de la JVM et des imports

## Versions TweetyProject

| Version | Date | Nouveautés | Notes |
|---------|------|------------|-------|
| **1.28** | Janvier 2025 | arg.caf, k-admissibility | Version stable recommandée |
| **1.29** | Juillet 2025 | arg.eaf (Epistemic AF), graph rendering | Dernière version |

> **Configuration**: La version peut être changée dans la cellule de téléchargement des JARs (section 1.4).

## Limitations connues (Tweety 1.28/1.29)

| Limitation | Impact | Contournement |
|------------|--------|---------------|
| **CrMas/InformationObject** | Révision multi-agents échoue | API refactorisée, non disponible |
| **SimpleMlReasoner** | Bloque indéfiniment | Installer SPASS externe |
| **AF Learning** | ClassCastException | Bug interne, section désactivée |
| **ADF natif SAT** | Raisonnement ADF échoue | Solveur SAT natif requis (non JPype) |
| **FOL égalité** | Heap space | Utiliser EProver externe |

## Série complète Tweety (10 notebooks)

| # | Notebook | Thème | Prérequis |
|---|----------|-------|-----------|
| 1 | **Ce notebook** | Configuration JVM/JPype | - |
| 2 | [Tweety-2-Basic-Logics](Tweety-2-Basic-Logics.ipynb) | Logique Propositionnelle, FOL | Setup |
| 3 | [Tweety-3-Advanced-Logics](Tweety-3-Advanced-Logics.ipynb) | DL, Modale, QBF, Conditional | Setup |
| 4 | [Tweety-4-Belief-Revision](Tweety-4-Belief-Revision.ipynb) | CrMas, MUS, MaxSAT | Setup |
| 5 | [Tweety-5-Abstract-Argumentation](Tweety-5-Abstract-Argumentation.ipynb) | Dung AF, CF2, Génération | Setup |
| 6 | [Tweety-6-Structured-Argumentation](Tweety-6-Structured-Argumentation.ipynb) | ASPIC+, DeLP, ABA, ASP | Setup + Clingo |
| 7a | [Tweety-7a-Extended-Frameworks](Tweety-7a-Extended-Frameworks.ipynb) | ADF, Bipolar, WAF, SAF, SetAF | Setup |
| 7b | [Tweety-7b-Ranking-Probabilistic](Tweety-7b-Ranking-Probabilistic.ipynb) | Ranking Semantics, Probabiliste | Setup |
| 8 | [Tweety-8-Agent-Dialogues](Tweety-8-Agent-Dialogues.ipynb) | Agents, Dialogues argumentatifs | Setup |
| 9 | [Tweety-9-Preferences](Tweety-9-Preferences.ipynb) | Préférences, Théorie du vote | Setup |

> **Note**: Le notebook original Tweety-7 est conservé pour référence historique.

---

## Plan de ce Notebook

**Section 1 : [Installation et Configuration](#partie1)**
* 1.2 [Pré-requis](#1.2)
* 1.3 [Installation des Packages Python](#1.3)
* 1.4 [Téléchargement des JARs Tweety](#1.4)
* 1.5 [Configuration des Outils Externes](#1.5)
* 1.6 [Démarrage de la JVM](#1.6)
* 1.7 [Concepts Clés](#1.7)

---

## Partie 1 : Introduction et Configuration
<a id="partie1"></a>

Cette section couvre la mise en place de l'environnement nécessaire pour exécuter les exemples de ce notebook.

### 1.2 Pré-requis
<a id="1.2"></a>

*   **Python 3.x** : Assurez-vous d'avoir une installation Python fonctionnelle (testé avec 3.10+).
*   **Java Development Kit (JDK)** : JPype nécessite un JDK (version 11 ou supérieure recommandée) pour démarrer la JVM.
    *   Vérifiez votre installation avec `java -version` dans un terminal.
    *   Assurez-vous que la variable d'environnement `JAVA_HOME` pointe vers le répertoire racine de votre installation JDK (par exemple, `/usr/lib/jvm/java-17-openjdk-amd64` sur Linux, `C:\Program Files\Java\jdk-17` sur Windows).
    *   Si `JAVA_HOME` n'est pas définie, le script de démarrage de la JVM (section 1.6) essaiera de trouver un JDK automatiquement, mais il est préférable de la définir explicitement.
*   **(Optionnel) Outils Externes** : Pour certaines fonctionnalités avancées (voir section [1.5](#1.5)), vous devrez installer des outils spécifiques et configurer leurs chemins d'accès plus loin dans le notebook.
*   **(Optionnel) Graphviz**: Pour visualiser certains graphes d'argumentation (non implémenté dans ce tutoriel mais possible).

### 1.3 Installation des Packages Python
<a id="1.3"></a>

Nous utilisons `jpype1` pour le pont Java-Python. Les autres packages (`requests`, `tqdm`) sont des utilitaires pour le téléchargement des JARs.

In [14]:
# Vérification/Installation des packages Python nécessaires
import importlib
import sys
import subprocess

# Le package s'appelle 'jpype1' sur PyPI, mais on importe 'jpype'
packages_to_check = {'jpype': 'jpype1', 'requests': 'requests', 'tqdm': 'tqdm', 'clingo': 'clingo'}
packages_to_install = []
all_found = True

print("--- Vérification des packages Python requis ---")
for import_name, install_name in packages_to_check.items():
    try:
        importlib.import_module(import_name)
        print(f"✔️ {import_name} trouvé.")
    except ImportError:
        print(f"⚠️ {import_name} manquant (package: {install_name}).")
        packages_to_install.append(install_name)
        all_found = False

if packages_to_install:
    print(f"\nTentative d'installation des packages manquants: {', '.join(packages_to_install)}...")
    try:
        # Utiliser -q pour une sortie moins verbeuse, retirer si le debug est nécessaire
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + packages_to_install)
        print(f"✅ Packages {', '.join(packages_to_install)} installés.")
        print("\n‼️ IMPORTANT : Vous devez probablement REDÉMARRER LE NOYAU (Kernel -> Restart Kernel) pour que les nouveaux packages soient pris en compte.")
        # Marquer comme non trouvés pour l'instant, car le noyau doit redémarrer
        all_found = False
    except Exception as e:
        print(f"❌ Échec de l'installation automatique : {e}")
        print(f"   Veuillez installer manuellement : pip install {' '.join(packages_to_install)}")
        all_found = False # Échec -> pas trouvé

if all_found:
    print("\n✔️ Tous les packages Python requis sont présents.")

# Importations pour vérifier après redémarrage potentiel (ne lèvera pas d'erreur ici)
try:
    import jpype
    import requests
    import tqdm
except ImportError:
    if not packages_to_install: # Si on n'a pas essayé d'installer, c'est une autre erreur
         print("\n⚠️ Erreur d'import inattendue. Vérifiez votre environnement Python.")
    # Si on a installé, le message d'erreur est déjà affiché.
    pass

--- Vérification des packages Python requis ---
✔️ jpype trouvé.
✔️ requests trouvé.
✔️ tqdm trouvé.
✔️ clingo trouvé.

✔️ Tous les packages Python requis sont présents.


### 1.4 Téléchargement des JARs Tweety et Dépendances
<a id="1.4"></a>

TweetyProject est une collection de bibliothèques Java distribuées sous forme de fichiers JAR. Pour utiliser Tweety avec JPype, nous devons télécharger ces JARs et les rendre accessibles à la JVM.

Le script suivant va :
1.  Créer un sous-dossier `libs/` s'il n'existe pas.
2.  Vérifier si l'URL de base pour la version spécifiée de Tweety (`TWEETY_VERSION`) est accessible.
3.  Télécharger (ou vérifier la présence) du JAR **principal** `tweety-full-...-with-dependencies.jar`. Celui-ci contient le noyau de Tweety et de nombreuses dépendances courantes.
4.  Télécharger (ou vérifier la présence) des JARs **spécifiques aux modules** utilisés dans ce notebook (argumentation, logiques spécifiques, etc.). La liste `REQUIRED_MODULES` définit les modules nécessaires.

*Note : Le téléchargement peut prendre quelques minutes lors de la première exécution. Assurez-vous que tous les JARs nécessaires sont bien présents dans le dossier `libs/` avant de démarrer la JVM, car des JARs manquants causeront des erreurs d'import plus loin.*

In [15]:
import pathlib
import urllib.request
import os
import requests
from tqdm.auto import tqdm # Barre de progression plus jolie dans les notebooks

# --- Configuration TweetyProject ---
# Versions disponibles:
#   - 1.28 (Janvier 2025): Version stable avec arg.caf, k-admissibility
#   - 1.29 (Juillet 2025): Ajoute arg.eaf (Epistemic AF), graph rendering, bugfixes mineurs
#
# Limitations connues (présentes dans 1.28, non confirmées corrigées en 1.29):
#   - CrMas/InformationObject: API refactorisée, certaines classes manquantes
#   - AF Learning: ClassCastException interne
#   - SimpleMlReasoner: Bloque sans SPASS externe
#   - ADF: Nécessite solveur SAT natif (Minisat) non disponible via JPype
#
# Recommandation: Utiliser 1.28 pour la stabilité, 1.29 pour arg.eaf ou les bugfixes
TWEETY_VERSION = "1.29"  # Changer en "1.29" pour utiliser la dernière version

BASE_URL = f"https://tweetyproject.org/builds/{TWEETY_VERSION}/"
LIB_DIR = pathlib.Path("libs")
LIB_DIR.mkdir(exist_ok=True)
CORE_JAR_NAME = f"org.tweetyproject.tweety-full-{TWEETY_VERSION}-with-dependencies.jar"
CORE_JAR_PATH = LIB_DIR / CORE_JAR_NAME

# Modules Tweety requis (couvre tous les exemples de la série)
# Note: arg.eaf (Epistemic AF) est uniquement disponible en 1.29+
REQUIRED_MODULES_BASE = sorted([
    "arg.adf", "arg.aba", "arg.bipolar", "arg.aspic", "arg.dung", "arg.weighted",
    "arg.social", "arg.setaf", "arg.rankings", "arg.prob", "arg.extended",
    "arg.delp", "arg.deductive", "arg.caf",
    "beliefdynamics", "agents.dialogues", "action", "preferences",
    "logics.pl", "logics.fol", "logics.ml", "logics.dl", "logics.cl",
    "logics.qbf", "logics.pcl", "logics.rcl", "logics.rpcl", "logics.mln", "logics.bpm",
    "lp.asp",
    "math", "commons", "agents"
])

# Ajouter arg.eaf pour version 1.29+
REQUIRED_MODULES = REQUIRED_MODULES_BASE.copy()
if TWEETY_VERSION >= "1.29":
    REQUIRED_MODULES.append("arg.eaf")
    REQUIRED_MODULES = sorted(REQUIRED_MODULES)
    print(f"Version {TWEETY_VERSION}: Module arg.eaf (Epistemic AF) inclus.")

# Vérification accessibilité URL de base
print(f"Vérification de l'accès à {BASE_URL}...")
url_accessible = False
try:
    response = requests.head(BASE_URL, timeout=10)
    response.raise_for_status()
    print(f"✔️ URL de base Tweety v{TWEETY_VERSION} accessible.")
    url_accessible = True
except requests.exceptions.RequestException as e:
    print(f"❌ Impossible d'accéder à l'URL de base {BASE_URL}. Vérifiez la version ou votre connexion. Erreur : {e}")
    print("   Le téléchargement des JARs manquants échouera. Seuls les JARs locaux seront utilisables.")

# Barre de progression TQDM
class TqdmUpTo(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

# Fonction de téléchargement
def download_jar(jar_name, url_base, target_dir):
    jar_path = target_dir / jar_name
    url = url_base + jar_name
    newly_downloaded = False

    if not jar_path.exists():
        if not url_accessible:
            print(f"   ⏭️ '{jar_name}' manquant, mais URL inaccessible. Téléchargement sauté.")
            return False, False
        try:
            response = requests.head(url, timeout=5)
            response.raise_for_status()
            with TqdmUpTo(unit='B', unit_scale=True, unit_divisor=1024, miniters=1, desc=jar_name[:40]) as t:
                urllib.request.urlretrieve(url, filename=jar_path, reporthook=t.update_to)
            if jar_path.exists() and jar_path.stat().st_size > 0:
                newly_downloaded = True
            else:
                print(f"\n❓ Téléchargement de {jar_name} terminé mais fichier vide ou absent.")
                if jar_path.exists(): jar_path.unlink(missing_ok=True)
        except requests.exceptions.HTTPError as e:
            print(f"\n❌ {jar_name} non trouvé sur le serveur (Erreur {e.response.status_code}).")
            return False, False
        except Exception as e:
            print(f"\n❌ Erreur lors du téléchargement de {jar_name}: {e}")
            if jar_path.exists(): jar_path.unlink(missing_ok=True)
            return False, False

    return newly_downloaded, jar_path.exists()

# --- Exécution du Téléchargement ---

# 1. JAR Core
print(f"\n--- Vérification/Téléchargement JAR Core ---")
core_new, core_exists = download_jar(CORE_JAR_NAME, BASE_URL, LIB_DIR)
if core_exists:
    status = "téléchargé" if core_new else "déjà présent"
    print(f"✔️ JAR Core '{CORE_JAR_NAME}' {status}.")
else:
     print(f"❌ ERREUR CRITIQUE : Le JAR core {CORE_JAR_NAME} est manquant et n'a pas pu être téléchargé.")

# 2. JARs Modules
print(f"\n--- Vérification/Téléchargement des {len(REQUIRED_MODULES)} JARs de modules ---")
modules_downloaded_count = 0
modules_present_count = 0
modules_missing = []

if core_exists:
    with tqdm(REQUIRED_MODULES, desc="Modules") as pbar:
        for module in pbar:
            pbar.set_postfix_str(module, refresh=True)
            module_jar_name = f"org.tweetyproject.{module}-{TWEETY_VERSION}-with-dependencies.jar"
            new_dl, present = download_jar(module_jar_name, BASE_URL, LIB_DIR)
            if new_dl:
                modules_downloaded_count += 1
            if present:
                modules_present_count += 1
            else:
                modules_missing.append(module_jar_name)
else:
    print("\nSkipping module download because Core JAR is missing.")
    modules_present_count = 0
    for module in REQUIRED_MODULES:
         module_jar_name = f"org.tweetyproject.{module}-{TWEETY_VERSION}-with-dependencies.jar"
         if (LIB_DIR / module_jar_name).exists():
              modules_present_count += 1
         else:
              modules_missing.append(module_jar_name)


print(f"\n--- Résumé Téléchargement ---")
print(f"  Version Tweety: {TWEETY_VERSION}")
print(f"  JAR Core: {'Présent' if core_exists else 'MANQUANT'}")
print(f"  JARs Modules:")
print(f"    - Nouveaux téléchargés : {modules_downloaded_count}")
print(f"    - Total présents       : {modules_present_count} / {len(REQUIRED_MODULES)}")
if modules_missing:
    print(f"    - ⚠️ Modules MANQUANTS : {', '.join(modules_missing)}")
print(f"  Chemin du dossier libs : {LIB_DIR.resolve()}")

# Vérification finale
all_jars = list(LIB_DIR.glob("*.jar"))
if not core_exists or modules_present_count < len(REQUIRED_MODULES):
    print("\n⚠️ Des JARs Tweety semblent manquants. Vérifiez les messages d'erreur ci-dessus.")
    print("   Le démarrage de la JVM ou l'utilisation de certaines fonctionnalités échoueront probablement.")
else:
    print("\n✔️ Vérification des JARs Tweety terminée. Le démarrage de la JVM peut continuer.")

Version 1.29: Module arg.eaf (Epistemic AF) inclus.
Vérification de l'accès à https://tweetyproject.org/builds/1.29/...
✔️ URL de base Tweety v1.29 accessible.

--- Vérification/Téléchargement JAR Core ---
✔️ JAR Core 'org.tweetyproject.tweety-full-1.29-with-dependencies.jar' déjà présent.

--- Vérification/Téléchargement des 34 JARs de modules ---


Modules:   0%|          | 0/34 [00:00<?, ?it/s]


--- Résumé Téléchargement ---
  Version Tweety: 1.29
  JAR Core: Présent
  JARs Modules:
    - Nouveaux téléchargés : 0
    - Total présents       : 34 / 34
  Chemin du dossier libs : D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\Tweety\libs

✔️ Vérification des JARs Tweety terminée. Le démarrage de la JVM peut continuer.


#### Fichiers de Données Complémentaires

En plus des JARs, TweetyProject utilise des fichiers de données d'exemple pour les différentes logiques et systèmes d'argumentation :

- **DeLP** : `birds.txt`, `nixon.txt` (exemples de raisonnement défaisable)
- **ABA** : `example1.aba`, `example2.aba` (Assumption-Based Argumentation)
- **ASPIC+** : `ex1.aspic` (règles strictes et défaisables)
- **Logiques** : fichiers `.proplogic`, `.fologic`, `.mlogic`, `.qbf`

Ces fichiers sont téléchargés depuis le dépôt GitHub officiel de TweetyProject.

In [16]:
# --- 1.4bis Téléchargement des Fichiers de Données (.txt, .aba, etc.) ---
print("\n--- 1.4bis Téléchargement des Fichiers de Données ---")

import pathlib
import requests
import os
from tqdm.auto import tqdm

# Créer le dossier de destination s'il n'existe pas
RESOURCE_DIR = pathlib.Path("resources")
RESOURCE_DIR.mkdir(exist_ok=True)
print(f"ℹ️ Les fichiers de données seront téléchargés dans: {RESOURCE_DIR.resolve()}")

# Base URL pour les fichiers bruts sur GitHub (branche main)
GITHUB_RAW_BASE_URL = "https://raw.githubusercontent.com/TweetyProjectTeam/TweetyProject/main/"

# Mapper les noms de fichiers aux chemins relatifs probables dans le dépôt GitHub
# (Liste inchangée par rapport à votre version)
FILES_TO_DOWNLOAD = {
    # DeLP Files
    "birds.txt": "org-tweetyproject-arg-delp/src/main/resources/birds.txt",
    "birds2.txt": "org-tweetyproject-arg-delp/src/main/resources/birds2.txt",
    "nixon.txt": "org-tweetyproject-arg-delp/src/main/resources/nixon.txt",
    "counterarg.txt": "org-tweetyproject-arg-delp/src/main/resources/counterarg.txt",
    # ABA Files
    "example1.aba": "org-tweetyproject-arg-aba/src/main/resources/example1.aba",
    "example2.aba": "org-tweetyproject-arg-aba/src/main/resources/example2.aba",
    "example5.aba": "org-tweetyproject-arg-aba/src/main/resources/example5.aba",
    "smp_fol.aba": "org-tweetyproject-arg-aba/src/main/resources/smp_fol.aba",
    # ASPIC Files
    "ex1.aspic": "org-tweetyproject-arg-aspic/src/main/resources/ex1.aspic",
    "ex5_fol.aspic": "org-tweetyproject-arg-aspic/src/main/resources/ex5_fol.aspic",
    # Autres Logiques/Args
     "examplebeliefbase.proplogic": "org-tweetyproject-logics-pl/src/main/resources/examplebeliefbase.proplogic",
     "examplebeliefbase_multiple.proplogic": "org-tweetyproject-logics-pl/src/main/resources/examplebeliefbase_multiple.proplogic",
     "examplebeliefbase_xor.proplogic": "org-tweetyproject-logics-pl/src/main/resources/examplebeliefbase_xor.proplogic",
     "dimacs_ex4.cnf": "org-tweetyproject-logics-pl/src/main/resources/dimacs_ex4.cnf",
     "examplebeliefbase.dlogic": "org-tweetyproject-logics-dl/src/main/resources/examplebeliefbase.dlogic",
     "examplebeliefbase2.fologic": "org-tweetyproject-logics-fol/src/main/resources/examplebeliefbase2.fologic",
     "examplebeliefbase.mlogic": "org-tweetyproject-logics-ml/src/main/resources/examplebeliefbase.mlogic",
     "examplebeliefbase2.mlogic": "org-tweetyproject-logics-ml/src/main/resources/examplebeliefbase2.mlogic",
     "tweety-example.qbf": "org-tweetyproject-logics-qbf/src/main/resources/tweety-example.qbf",
     "qdimacs-example1.qdimacs": "org-tweetyproject-logics-qbf/src/main/resources/qdimacs-example1.qdimacs",
     "qcir-example1.qcir": "org-tweetyproject-logics-qbf/src/main/resources/qcir-example1.qcir",
     "qcir-example2-sat.qcir": "org-tweetyproject-logics-qbf/src/main/resources/qcir-example2-sat.qcir",
     # BPMN - Gardons celui de votre TP comme exemple
     "problematic_hit.bpmn": "org-tweetyproject-logics-bpm/src/main/resources/problematic_hit.bpmn",
     "monkey.desc": "org-tweetyproject-action/src/main/resources/monkey.desc",
     "conditioner.desc": "org-tweetyproject-action/src/main/resources/conditioner.desc",
     "adf_example.txt": "org-tweetyproject-arg-adf/src/main/resources/adf_example.txt",
     "ex1.apx": "org-tweetyproject-arg-setaf/src/main/resources/ex1.apx",
     "ex1_bonzon.apx": "org-tweetyproject-arg-rankings/src/main/resources/ex1_bonzon.apx",
     "ex1.tgf": "org-tweetyproject-arg-dung/src/main/resources/ex1.tgf",
}

# Barre de progression TQDM (inchangée)
class TqdmUpTo(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None: self.total = tsize
        self.update(b * bsize - self.n)

# Boucle de téléchargement
downloaded_count = 0
present_count = 0
failed_count = 0
skipped_count = 0

print(f"\nVérification/Téléchargement de {len(FILES_TO_DOWNLOAD)} fichiers de données...")

# Vérification de l'accessibilité GitHub avec un fichier test (README.md)
github_accessible = False
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
try:
    # Tester avec README.md car la racine n'est pas accessible sur raw.githubusercontent.com
    test_url = GITHUB_RAW_BASE_URL + "README.md"
    response_gh = requests.head(test_url, timeout=10, headers=headers, allow_redirects=True)
    github_accessible = response_gh.ok
    if not github_accessible:
        print(f"⚠️ Impossible d'accéder a GitHub {test_url}. Code: {response_gh.status_code}")
    else:
        print("✔️ Accès à la base GitHub Raw confirmé.")
except requests.exceptions.RequestException as e_gh:
    print(f"⚠️ Erreur de connexion à GitHub: {e_gh}")

if not github_accessible:
    print("   Skipping all file downloads from GitHub.")
    skipped_count = len(FILES_TO_DOWNLOAD)
    # Compter ceux déjà présents localement
    for filename in FILES_TO_DOWNLOAD:
        if (RESOURCE_DIR / filename).exists():
             present_count += 1
else:
    with tqdm(FILES_TO_DOWNLOAD.items(), desc="Fichiers Données") as pbar:
        for filename, relative_path in pbar:
            pbar.set_postfix_str(filename, refresh=True)
            target_path = RESOURCE_DIR / filename
            file_url = GITHUB_RAW_BASE_URL + relative_path

            if target_path.exists() and target_path.stat().st_size > 0:
                present_count += 1
                continue # Skip download if already present and not empty

            try:
                # Utiliser GET avec header pour le téléchargement aussi
                response = requests.get(file_url, stream=True, timeout=15, headers=headers) # Timeout augmenté
                # Vérifier explicitement 404
                if response.status_code == 404:
                    print(f"\n   ⚠️ Fichier '{filename}' non trouvé sur GitHub (404) à l'URL: {file_url}")
                    failed_count += 1
                    continue
                response.raise_for_status() # Lève une exception pour les autres erreurs HTTP

                total_size = int(response.headers.get('content-length', 0))
                with TqdmUpTo(unit='B', unit_scale=True, unit_divisor=1024, total=total_size, desc=filename, leave=False) as t:
                    with open(target_path, 'wb') as f:
                        for chunk in response.iter_content(chunk_size=8192):
                            if chunk:
                                f.write(chunk)
                                t.update(len(chunk))

                if target_path.exists() and target_path.stat().st_size > 0:
                    downloaded_count += 1
                    present_count += 1
                else:
                    print(f"\n   ❓ Téléchargement de '{filename}' terminé mais fichier vide/absent.")
                    if target_path.exists(): target_path.unlink(missing_ok=True)
                    failed_count += 1

            except requests.exceptions.RequestException as e:
                print(f"\n   ❌ Échec téléchargement '{filename}' : {e}")
                if target_path.exists(): target_path.unlink(missing_ok=True)
                failed_count += 1
            except Exception as e_other:
                 print(f"\n   ❌ Erreur inattendue pour '{filename}': {e_other}")
                 if target_path.exists(): target_path.unlink(missing_ok=True)
                 failed_count += 1


# Résumé
print(f"\n--- Résumé Téléchargement Données ---")
print(f"   Dossier cible          : {RESOURCE_DIR.resolve()}")
print(f"   Fichiers téléchargés  : {downloaded_count}")
print(f"   Fichiers déjà présents : {present_count - downloaded_count}")
if skipped_count > 0: print(f"   Fichiers sautés (connexion échouée): {skipped_count}")
print(f"   Total fichiers OK      : {present_count} / {len(FILES_TO_DOWNLOAD)}")
if failed_count > 0:
     print(f"   ⚠️ Échecs (Non trouvés sur GitHub ou autre erreur): {failed_count}")
     print("      Les sections utilisant ces fichiers pourraient échouer.")
elif skipped_count == 0 and present_count == len(FILES_TO_DOWNLOAD):
     print("✔️ Tous les fichiers de données nécessaires semblent présents.")


--- 1.4bis Téléchargement des Fichiers de Données ---
ℹ️ Les fichiers de données seront téléchargés dans: D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\Tweety\resources

Vérification/Téléchargement de 29 fichiers de données...
✔️ Accès à la base GitHub Raw confirmé.


Fichiers Données:   0%|          | 0/29 [00:00<?, ?it/s]


--- Résumé Téléchargement Données ---
   Dossier cible          : D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\Tweety\resources
   Fichiers téléchargés  : 0
   Fichiers déjà présents : 29
   Total fichiers OK      : 29 / 29
✔️ Tous les fichiers de données nécessaires semblent présents.


### 1.5 Configuration des Outils Externes (Optionnel)
<a id="1.5"></a>

Certaines fonctionnalités avancées de Tweety s'appuient sur des outils externes (solveurs SAT/MaxSAT, prouveurs FOL/ML, énumérateurs MUS, solveurs ASP). Pour utiliser ces fonctionnalités, vous devez :

1.  **Installer l'outil externe** séparément en suivant les instructions de son site web.
2.  **Indiquer le chemin d'accès** à l'exécutable de l'outil dans la cellule de code ci-dessous.

**Outils potentiellement utilisés dans ce notebook :**

*   **Solveurs SAT externes** (pour `logics.pl.sat.CmdLineSatSolver` - Section 2.1.3) :
    *   Lingeling, CaDiCaL, Kissat, MiniSat, Glucose, etc. (prennent souvent le format DIMACS)
*   **Prouveur FOL** (pour `logics.fol.reasoner.EFOLReasoner` - Section 2.2.2) :
    *   **EProver** : Téléchargement et compilation peuvent être complexes. ([Site Eprover](https://eprover.org/))
    *   *Alternative* : Tweety peut aussi utiliser SPASS pour certains fragments FOL, configuré ci-dessous.
*   **Prouveur ML** (pour `logics.ml.reasoner.SPASSMlReasoner` - Section 2.4.2) :
    *   **SPASS** : ([Site SPASS](https://www.spass-prover.org/)) - Binaires souvent disponibles.
*   **Énumérateur MUS** (pour `logics.pl.sat.MarcoMusEnumerator` - Section 3.3) :
    *   **MARCO** : Script Python. ([Lien potentiel via recherche](https://github.com/marcomus/marco))
*   **Solveur MaxSAT** (pour `logics.pl.sat.OpenWboSolver` - Section 3.4) :
    *   **Open-WBO** : ([Site Open-WBO](https://github.com/sat-group/open-wbo))
*   **Solveur ASP** (pour `lp.asp.reasoner.ClingoSolver` - Section 4.6) :
    *   **Clingo** (partie de Potassco) : ([Site Potassco](https://potassco.org/))
*   **Solveurs ADF** (pour `arg.adf.sat.solver.*` - Section 5.1) :
    *   PicoSAT, Lingeling, MiniSat (fournis avec Tweety pour certaines plateformes, voir `libs/`)

**Instructions :**
*   Modifiez les variables `*_PATH` dans la cellule suivante avec les chemins corrects sur **votre** système.
*   Si un outil n'est pas installé ou si vous ne souhaitez pas l'utiliser, laissez le chemin vide (`""`) ou commentez la ligne correspondante. Le notebook essaiera d'utiliser des alternatives internes (plus lentes) ou sautera les sections concernées.
*   Sous Windows, n'oubliez pas l'extension `.exe` et utilisez des anti-slashs (`\\`) ou des slashs (`/`) pour les chemins.

In [17]:
# --- Configuration Outils Externes (avec auto-download) ---
import os
import pathlib
import platform
import shutil
import subprocess
import urllib.request
import zipfile
import stat

# Repertoire pour les outils externes
EXT_TOOLS_DIR = pathlib.Path("ext_tools")
EXT_TOOLS_DIR.mkdir(exist_ok=True)

# Repertoire pour les bibliotheques natives SAT (pour ADF)
# Ces DLLs/SO sont utilisees par NativeMinisatSolver, NativeLingelingSolver, etc.
NATIVE_LIBS_DIR = pathlib.Path("../libs/native")  # Chemin relatif depuis Tweety/
if not NATIVE_LIBS_DIR.exists():
    # Essayer chemin alternatif
    alt_paths = [
        pathlib.Path("libs/native"),
        pathlib.Path("../../libs/native"),
    ]
    for alt in alt_paths:
        if alt.exists():
            NATIVE_LIBS_DIR = alt
            break

# Dictionnaire pour stocker les chemins des outils externes
EXTERNAL_TOOLS = {
    "SAT_SOLVER": "",
    "EPROVER": "",
    "MARCO": "",
    "OPEN_WBO": "",
    "CLINGO": "",
    "SPASS": ""
}

def get_tool_path(tool_name):
    """Retourne le chemin valide d'un outil ou None."""
    path_str = EXTERNAL_TOOLS.get(tool_name, "")
    if not path_str: 
        return None
    if shutil.which(path_str):
        return path_str
    path_obj = pathlib.Path(path_str)
    if path_obj.is_file() and os.access(path_obj, os.R_OK):
        return str(path_obj.resolve())
    return None

system = platform.system()

# === 1. Configuration de Clingo ===
print("=== 1. Configuration de Clingo ===")

clingo_dir = EXT_TOOLS_DIR / "clingo"
clingo_dir.mkdir(exist_ok=True)
clingo_exe = clingo_dir / ("clingo.exe" if system == "Windows" else "clingo")

# Verifier si clingo est deja present
clingo_in_path = shutil.which("clingo") or shutil.which("clingo.exe")
if clingo_in_path:
    EXTERNAL_TOOLS["CLINGO"] = clingo_in_path
    print(f"  OK Clingo trouve dans PATH: {clingo_in_path}")
elif clingo_exe.exists():
    EXTERNAL_TOOLS["CLINGO"] = str(clingo_exe.resolve())
    print(f"  OK Clingo deja present: {clingo_exe}")
else:
    # Telecharger automatiquement
    print("  Telechargement de Clingo...")
    clingo_version = "5.4.0"  # Version stable avec binaires disponibles
    
    if system == "Windows":
        clingo_url = f"https://github.com/potassco/clingo/releases/download/v{clingo_version}/clingo-{clingo_version}-win64.zip"
        clingo_archive = clingo_dir / "clingo.zip"
        
        try:
            print(f"  Depuis {clingo_url}")
            urllib.request.urlretrieve(clingo_url, clingo_archive)
            
            # Extraire
            with zipfile.ZipFile(clingo_archive, 'r') as zip_ref:
                zip_ref.extractall(clingo_dir)
            
            # Trouver l'executable
            for exe in clingo_dir.rglob("clingo.exe"):
                shutil.move(str(exe), str(clingo_exe))
                EXTERNAL_TOOLS["CLINGO"] = str(clingo_exe.resolve())
                print(f"  OK Clingo installe: {clingo_exe}")
                break
            
            # Nettoyer
            clingo_archive.unlink(missing_ok=True)
            for d in clingo_dir.glob("clingo-*"):
                if d.is_dir():
                    shutil.rmtree(d, ignore_errors=True)
                    
        except Exception as e:
            print(f"  Erreur telechargement Clingo: {e}")
            print("  Installez manuellement depuis https://github.com/potassco/clingo/releases")
    
    elif system == "Linux":
        clingo_url = f"https://github.com/potassco/clingo/releases/download/v{clingo_version}/clingo-{clingo_version}-linux-x86_64.tar.gz"
        clingo_archive = clingo_dir / "clingo.tar.gz"
        
        try:
            import tarfile
            urllib.request.urlretrieve(clingo_url, clingo_archive)
            
            with tarfile.open(clingo_archive, "r:gz") as tar:
                tar.extractall(path=clingo_dir)
            
            for exe in clingo_dir.rglob("clingo"):
                if exe.is_file():
                    shutil.move(str(exe), str(clingo_exe))
                    os.chmod(clingo_exe, stat.S_IRWXU | stat.S_IRGRP | stat.S_IXGRP)
                    EXTERNAL_TOOLS["CLINGO"] = str(clingo_exe.resolve())
                    print(f"  OK Clingo installe: {clingo_exe}")
                    break
            
            clingo_archive.unlink(missing_ok=True)
        except Exception as e:
            print(f"  Erreur installation Clingo: {e}")
    else:
        print(f"  Installation auto Clingo non supportee pour {system}")

# === 2. Configuration de SPASS ===
print("=== 2. Configuration de SPASS ===")

spass_dir = EXT_TOOLS_DIR / "spass"
spass_dir.mkdir(exist_ok=True)
spass_exe = spass_dir / ("SPASS.exe" if system == "Windows" else "SPASS")

spass_in_path = shutil.which("SPASS") or shutil.which("SPASS.exe")
if spass_in_path:
    EXTERNAL_TOOLS["SPASS"] = spass_in_path
    print(f"  OK SPASS trouve dans PATH: {spass_in_path}")
elif spass_exe.exists():
    EXTERNAL_TOOLS["SPASS"] = str(spass_exe.resolve())
    print(f"  OK SPASS deja present: {spass_exe}")
else:
    if system == "Windows":
        print("  Telechargement de SPASS pour Windows...")
        spass_url = "https://www.spass-prover.org/download/binaries/spass30windows.exe"
        
        try:
            urllib.request.urlretrieve(spass_url, spass_exe)
            if spass_exe.exists():
                EXTERNAL_TOOLS["SPASS"] = str(spass_exe.resolve())
                print(f"  OK SPASS installe: {spass_exe}")
        except Exception as e:
            print(f"  Erreur telechargement SPASS: {e}")
            print("  Telechargez manuellement depuis https://www.spass-prover.org/download/")
    
    elif system == "Linux":
        arch = "64" if platform.architecture()[0] == "64bit" else "32"
        spass_url = f"https://www.spass-prover.org/download/binaries/spass35pclinux{arch}.tgz"
        spass_archive = spass_dir / "spass.tgz"
        
        try:
            import tarfile
            urllib.request.urlretrieve(spass_url, spass_archive)
            
            with tarfile.open(spass_archive, "r:gz") as tar:
                tar.extractall(path=spass_dir)
            
            extracted = spass_dir / "SPASS" / "SPASS"
            if extracted.exists():
                shutil.move(str(extracted), str(spass_exe))
                os.chmod(spass_exe, stat.S_IRWXU | stat.S_IRGRP | stat.S_IXGRP)
                EXTERNAL_TOOLS["SPASS"] = str(spass_exe.resolve())
                print(f"  OK SPASS installe: {spass_exe}")
            
            spass_archive.unlink(missing_ok=True)
            shutil.rmtree(spass_dir / "SPASS", ignore_errors=True)
        except Exception as e:
            print(f"  Erreur installation SPASS: {e}")
    else:
        print(f"  Installation auto SPASS non supportee pour {system}")

# === 3. Configuration de EProver (prouveur FOL) ===
print("=== 3. Configuration de EProver ===")

# EProver est fourni dans le depot (restaure depuis git history)
eprover_search_paths = [
    pathlib.Path("../ext_tools/EProver"),  # Depuis Tweety/
    pathlib.Path("ext_tools/EProver"),
    pathlib.Path("../../ext_tools/EProver"),
]
eprover_exe_name = "eprover.exe" if system == "Windows" else "eprover"

eprover_in_path = shutil.which("eprover") or shutil.which("eprover.exe")
if eprover_in_path:
    EXTERNAL_TOOLS["EPROVER"] = eprover_in_path
    print(f"  OK EProver trouve dans PATH: {eprover_in_path}")
else:
    for search_path in eprover_search_paths:
        eprover_exe = search_path / eprover_exe_name
        if eprover_exe.exists():
            EXTERNAL_TOOLS["EPROVER"] = str(eprover_exe.resolve())
            print(f"  OK EProver trouve: {eprover_exe.resolve()}")
            break
    else:
        print("  EProver non trouve dans les chemins standards.")
        print("  Telecharger depuis https://eprover.org/ ou utiliser les binaires du depot.")

# === 4. Configuration des bibliotheques SAT natives ===
print("=== 4. Configuration des bibliotheques SAT natives ===")

if NATIVE_LIBS_DIR.exists():
    print(f"  Repertoire libs/native trouve: {NATIVE_LIBS_DIR.resolve()}")
    
    # Verifier les DLLs/SO presentes
    sat_libs_found = []
    for solver in ["lingeling", "minisat", "picosat"]:
        dll_path = NATIVE_LIBS_DIR / f"{solver}.dll"
        so_path = NATIVE_LIBS_DIR / f"{solver}.so"
        if dll_path.exists():
            sat_libs_found.append(f"{solver}.dll")
        elif so_path.exists():
            sat_libs_found.append(f"{solver}.so")
    
    if sat_libs_found:
        print(f"  OK Bibliotheques SAT trouvees: {', '.join(sat_libs_found)}")
        print(f"     Ces libs seront ajoutees au java.library.path lors du demarrage JVM.")
    else:
        print("  Aucune bibliotheque SAT native trouvee dans libs/native/")
else:
    print(f"  Repertoire libs/native non trouve.")
    print("  Les solveurs SAT natifs (NativeMinisatSolver, etc.) ne seront pas disponibles.")
    print("  Seul Sat4jSolver sera utilisable (plus lent mais fonctionnel).")

# === 5. Configuration de SAT_SOLVER (Sat4j - Pure Java) ===
print("=== 5. Configuration de SAT_SOLVER ===")

# Sat4j est integre dans Tweety (pure Java), pas d'executable externe requis
EXTERNAL_TOOLS["SAT_SOLVER"] = "Sat4j (integre)"
print("  OK Sat4j disponible (integre dans les JARs Tweety)")
print("     Classe: org.tweetyproject.logics.pl.sat.Sat4jSolver")
print("     Note: Plus lent que les solveurs natifs mais multi-plateforme")

# === 6. Configuration de MARCO (MUS enumerator) ===
print("=== 6. Configuration de MARCO ===")

# Chercher dans plusieurs emplacements possibles
marco_search_paths = [
    pathlib.Path("../ext_tools/marco.py"),  # Depuis Tweety/ vers SymbolicAI/ext_tools/
    pathlib.Path("ext_tools/marco.py"),
    pathlib.Path("../../ext_tools/marco.py"),
]

marco_script = None
for mp in marco_search_paths:
    if mp.exists():
        marco_script = mp.resolve()
        break

if marco_script:
    EXTERNAL_TOOLS["MARCO"] = str(marco_script)
    print(f"  OK MARCO script trouve: {marco_script}")
    
    # Verifier Z3 Python
    try:
        import z3
        z3_version = z3.get_version_string()
        print(f"  OK Z3 Python bindings disponibles (version {z3_version})")
    except ImportError:
        print("  Attention: z3-solver package requis")
        print("             Installez avec: pip install z3-solver")
else:
    print(f"  MARCO script non trouve dans les chemins standards")
    print("  Les fonctionnalites MUS (MarcoMusEnumerator) ne seront pas disponibles")

# === 7. Configuration de OPEN_WBO (MaxSAT solver) ===
print("=== 7. Configuration de OPEN_WBO ===")

# Chercher dans plusieurs emplacements possibles
maxsat_search_paths = [
    pathlib.Path("../ext_tools/maxsat_solver.py"),  # Depuis Tweety/ vers SymbolicAI/ext_tools/
    pathlib.Path("ext_tools/maxsat_solver.py"),
    pathlib.Path("../../ext_tools/maxsat_solver.py"),
]

maxsat_script = None
for mp in maxsat_search_paths:
    if mp.exists():
        maxsat_script = mp.resolve()
        break

if maxsat_script:
    EXTERNAL_TOOLS["OPEN_WBO"] = str(maxsat_script)
    print(f"  OK MaxSAT solver script trouve: {maxsat_script}")
    
    # Verifier PySAT
    try:
        import pysat
        pysat_version = pysat.__version__
        print(f"  OK python-sat package disponible (version {pysat_version})")
        print("     Utilise RC2 (MaxSAT Evaluation 2018 winner)")
    except ImportError:
        print("  Attention: python-sat package requis")
        print("             Installez avec: pip install python-sat")
else:
    print(f"  MaxSAT solver script non trouve dans les chemins standards")
    print("  Les fonctionnalites MaxSAT (OpenWboSolver) ne seront pas disponibles")

# === 8. Resume ===
print("\n=== Resume des outils externes ===")
for tool, path in EXTERNAL_TOOLS.items():
    if tool == "SAT_SOLVER":
        # Cas special pour Sat4j (integre)
        print(f"  {tool:<12}: OK (Sat4j integre)")
    else:
        valid = get_tool_path(tool)
        status = "OK" if valid else "Non configure"
        print(f"  {tool:<12}: {status} ({path or '-'})")

print(f"\n  NATIVE_LIBS_DIR: {NATIVE_LIBS_DIR.resolve() if NATIVE_LIBS_DIR.exists() else 'Non trouve'}")
print("  EXTERNAL_TOOLS et get_tool_path sont disponibles pour ce notebook.")

=== 1. Configuration de Clingo ===
  OK Clingo deja present: ext_tools\clingo\clingo.exe
=== 2. Configuration de SPASS ===
  OK SPASS deja present: ext_tools\spass\SPASS.exe
=== 3. Configuration de EProver ===
  OK EProver trouve: D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\ext_tools\EProver\eprover.exe
=== 4. Configuration des bibliotheques SAT natives ===
  Repertoire libs/native trouve: D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\libs\native
  OK Bibliotheques SAT trouvees: lingeling.dll, minisat.dll, picosat.dll
     Ces libs seront ajoutees au java.library.path lors du demarrage JVM.
=== 5. Configuration de SAT_SOLVER ===
  OK Sat4j disponible (integre dans les JARs Tweety)
     Classe: org.tweetyproject.logics.pl.sat.Sat4jSolver
     Note: Plus lent que les solveurs natifs mais multi-plateforme
=== 6. Configuration de MARCO ===
  OK MARCO script trouve: D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\ext_tools\marco.py
  Attention: z3-solver package requis
             Installez avec

### 1.6 Démarrage de la JVM via JPype
<a id="1.6"></a>

JPype permet à Python d'interagir directement avec le code Java. Nous devons démarrer une Machine Virtuelle Java (JVM) et lui indiquer où trouver les fichiers JAR de Tweety que nous venons de télécharger.

* Le code tente de trouver votre `JAVA_HOME`. Modifiez `java_home_path` si nécessaire dans la cellule suivante.
* Il construit le `classpath` Java en listant tous les `.jar` dans le dossier `libs/`.
* `jpype.startJVM()` lance la JVM. Si elle est déjà lancée (ex: après redémarrage du noyau), l'appel est ignoré.
* **Crucial**: `jpype.imports.registerDomain(...)` doit être appelé **après** le démarrage de la JVM pour permettre les imports courts (ex: `from org.tweetyproject...`).

In [18]:
# --- 1.6 Démarrage de la JVM via JPype ---
print("\n--- 1.6 Démarrage de la JVM via JPype ---")

# -------- RAPPEL IMPORTANT --------
print("\n‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️")
print("‼️ SI DES JARs ONT ÉTÉ TÉLÉCHARGÉS (Cellule 7), REDÉMARREZ LE NOYAU MAINTENANT ‼️")
print("‼️ Kernel -> Restart Kernel...                                         ‼️")
print("‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️")

import jpype
import jpype.imports
import os
import pathlib
import platform
import urllib.request
import zipfile
import shutil
from jpype.types import * # Nécessaire pour JString etc. plus tard
import stat

# --- Configuration JDK Portable ---
# URLs de téléchargement Zulu JDK 17 (Azul)
JDK_DOWNLOAD_URLS = {
    "Windows": "https://cdn.azul.com/zulu/bin/zulu17.50.19-ca-jdk17.0.11-win_x64.zip",
    "Linux": "https://cdn.azul.com/zulu/bin/zulu17.50.19-ca-jdk17.0.11-linux_x64.tar.gz",
    "Darwin": "https://cdn.azul.com/zulu/bin/zulu17.50.19-ca-jdk17.0.11-macosx_x64.zip"
}

def find_portable_jdk():
    """
    Localise automatiquement le JDK portable dans l'arborescence du projet.
    Recherche dans plusieurs emplacements standards.
    """
    print("🔍 Recherche JDK portable dans l'arborescence projet...")

    # Chemins de recherche prioritaires pour le JDK portable
    # Depuis Tweety.ipynb: ./jdk-17-portable ou ../Argument_Analysis/jdk-17-portable
    search_paths = [
        pathlib.Path("jdk-17-portable"),
        pathlib.Path("../jdk-17-portable"),
        pathlib.Path("Argument_Analysis/jdk-17-portable"),
        pathlib.Path("../Argument_Analysis/jdk-17-portable"),
        pathlib.Path("../../jdk-17-portable"),
        pathlib.Path("MyIA.AI.Notebooks/SymbolicAI/jdk-17-portable"),
        pathlib.Path("MyIA.AI.Notebooks/SymbolicAI/Argument_Analysis/jdk-17-portable"),
    ]

    for base_path in search_paths:
        if base_path.exists():
            print(f"  Scan du répertoire: {base_path}")

            # Chercher des sous-dossiers JDK
            jdk_patterns = ["*jdk*", "zulu*", "*openjdk*", "*corretto*"]
            for pattern in jdk_patterns:
                jdk_dirs = list(base_path.glob(pattern))
                for jdk_dir in jdk_dirs:
                    if jdk_dir.is_dir():
                        # Vérifier présence de bin/java
                        exe_suffix = ".exe" if platform.system() == "Windows" else ""
                        java_exe = jdk_dir / "bin" / f"java{exe_suffix}"
                        if java_exe.exists():
                            print(f"✅ JDK portable trouvé: {jdk_dir.absolute()}")
                            return str(jdk_dir.absolute())

    print("⚠️ JDK portable non trouvé dans les chemins standards")
    return None

def download_portable_jdk():
    """
    Télécharge et extrait le JDK portable Zulu 17 automatiquement.
    Le JDK est installé dans ./jdk-17-portable/
    """
    system = platform.system()
    if system not in JDK_DOWNLOAD_URLS:
        print(f"❌ Système {system} non supporté pour le téléchargement automatique du JDK")
        return None

    url = JDK_DOWNLOAD_URLS[system]
    jdk_dir = pathlib.Path("jdk-17-portable")
    jdk_dir.mkdir(exist_ok=True)

    # Nom du fichier téléchargé
    archive_name = url.split("/")[-1]
    archive_path = jdk_dir / archive_name

    print(f"📥 Téléchargement du JDK portable depuis Azul...")
    print(f"   URL: {url}")
    print(f"   Destination: {archive_path}")

    try:
        # Télécharger avec barre de progression
        def download_progress(block_num, block_size, total_size):
            if total_size > 0:
                percent = min(100, block_num * block_size * 100 // total_size)
                if block_num % 100 == 0 or percent == 100:
                    print(f"\r   Progression: {percent}% ({block_num * block_size // (1024*1024)}MB)", end="", flush=True)

        urllib.request.urlretrieve(url, archive_path, download_progress)
        print()  # Nouvelle ligne après progression

        print(f"✅ Téléchargement terminé: {archive_path}")
        print(f"📦 Extraction de l'archive...")

        # Extraire l'archive
        if archive_name.endswith(".zip"):
            with zipfile.ZipFile(archive_path, 'r') as zip_ref:
                zip_ref.extractall(jdk_dir)
        elif archive_name.endswith(".tar.gz"):
            import tarfile
            with tarfile.open(archive_path, 'r:gz') as tar_ref:
                tar_ref.extractall(jdk_dir)

        # Supprimer l'archive après extraction
        archive_path.unlink()
        print(f"✅ Extraction terminée et archive supprimée")

        # Trouver le dossier JDK extrait
        extracted_jdk = find_portable_jdk()
        if extracted_jdk:
            print(f"✅ JDK portable installé: {extracted_jdk}")
            return extracted_jdk
        else:
            print("❌ JDK extrait mais non détecté - vérifiez le contenu de jdk-17-portable/")
            return None

    except Exception as e:
        print(f"❌ Erreur lors du téléchargement/extraction du JDK: {e}")
        return None

def find_java_home():
    """
    Trouve JAVA_HOME en vérifiant dans l'ordre:
    1. JDK portable existant
    2. Téléchargement automatique du JDK portable si manquant
    3. Variable d'environnement JAVA_HOME
    4. Détection automatique des JDKs système
    """
    # 1. Chercher d'abord un JDK portable existant
    portable_jdk = find_portable_jdk()
    if portable_jdk:
        print(f"🏠 Utilisation du JDK portable: {portable_jdk}")
        os.environ['JAVA_HOME'] = portable_jdk
        return portable_jdk

    # 2. Télécharger automatiquement le JDK portable
    print("\n📥 JDK portable non trouvé - tentative de téléchargement automatique...")
    downloaded_jdk = download_portable_jdk()
    if downloaded_jdk:
        os.environ['JAVA_HOME'] = downloaded_jdk
        return downloaded_jdk

    # 3. Vérifier la variable d'environnement JAVA_HOME
    java_home_env = os.getenv("JAVA_HOME")
    if java_home_env and pathlib.Path(java_home_env).is_dir():
        exe_suffix = ".exe" if platform.system() == "Windows" else ""
        if (pathlib.Path(java_home_env) / "bin" / f"java{exe_suffix}").exists():
            print(f"ℹ️ Utilisation de JAVA_HOME trouvé dans l'environnement : {java_home_env}")
            return java_home_env
        else:
            print(f"⚠️ JAVA_HOME ('{java_home_env}') trouvé mais ne semble pas être un JDK valide (bin/java manquant).")

    # 4. Détection automatique des JDKs système
    print("ℹ️ Tentative de détection automatique des JDKs système...")
    possible_locations = []
    if platform.system() == "Windows":
        java_dir = pathlib.Path("C:/Program Files/Java/")
        if java_dir.is_dir(): possible_locations = sorted(java_dir.glob("jdk-*/"), reverse=True)
        program_files_x86 = os.environ.get("ProgramFiles(x86)")
        if program_files_x86:
             java_dir_x86 = pathlib.Path(program_files_x86) / "Java"
             if java_dir_x86.is_dir(): possible_locations.extend(sorted(java_dir_x86.glob("jdk-*/"), reverse=True))
    elif platform.system() == "Linux":
         linux_paths = ["/usr/lib/jvm"]
         for p_str in linux_paths:
             p_obj = pathlib.Path(p_str)
             if p_obj.is_dir():
                 possible_locations.extend(sorted(p_obj.glob("java-*"), reverse=True))
                 possible_locations.extend(sorted(p_obj.glob("jdk*"), reverse=True))
    elif platform.system() == "Darwin": # macOS
        mac_path = pathlib.Path("/Library/Java/JavaVirtualMachines")
        if mac_path.is_dir():
            possible_locations.extend(sorted(mac_path.glob("jdk*.jdk"), reverse=True))

    exe_suffix = ".exe" if platform.system() == "Windows" else ""
    for p in possible_locations:
        java_bin_dir = p / "Contents/Home/bin" if platform.system() == "Darwin" else p / "bin"
        if java_bin_dir.is_dir() and (java_bin_dir / f"java{exe_suffix}").exists():
             detected_home = p / "Contents/Home" if platform.system() == "Darwin" else p
             print(f"✔️ JDK système valide détecté : {detected_home}")
             if not java_home_env:
                 print(f"   (Tentative de définition de JAVA_HOME pour ce script : {detected_home})")
                 os.environ['JAVA_HOME'] = str(detected_home)
             return str(detected_home)

    print("❌ ERREUR: JAVA_HOME n'est pas défini et aucun JDK n'a pu être détecté ou téléchargé.")
    print("           ==> Vérifiez votre connexion internet ou installez manuellement un JDK 17+ <==")
    return None

java_home_path = find_java_home()

# --- Construction Classpath ---
classpath_separator = os.pathsep
if 'LIB_DIR' not in globals(): LIB_DIR = pathlib.Path("libs")
jar_list = [str(p.resolve()) for p in LIB_DIR.glob("*.jar")]
classpath = ""
beliefdynamics_jar_found_in_glob = False
num_jars_found = len(jar_list)

if not jar_list:
    print("❌ ERREUR: Le dossier 'libs/' ne contient aucun fichier JAR. Le classpath est vide.")
else:
    classpath = classpath_separator.join(jar_list)
    print(f"\nClasspath ({num_jars_found} JARs trouvé(s)) assemblé.")
    print("\n--- DEBUG CLASSPATH ---")
    print(f"Longueur totale: {len(classpath)} caractères")
    beliefdynamics_jar_name = "org.tweetyproject.beliefdynamics"
    beliefdynamics_jar_full_path = None
    for jar_path in jar_list:
        if beliefdynamics_jar_name in jar_path:
             beliefdynamics_jar_found_in_glob = True
             beliefdynamics_jar_full_path = jar_path
             print(f"✔️ JAR '{beliefdynamics_jar_name}' TROUVÉ dans la liste: ...{jar_path[-80:]}")
             break
    if not beliefdynamics_jar_found_in_glob:
        print(f"⚠️⚠️⚠️ ATTENTION: Le JAR '{beliefdynamics_jar_name}' n'a PAS été trouvé par glob('*.jar') ! ⚠️⚠️⚠️")
    print("--- FIN DEBUG CLASSPATH ---\n")


# --- Démarrage JVM ---
jvm_started_in_this_cell = False
if not jpype.isJVMStarted():
    if not java_home_path:
        print("❌ Impossible de démarrer la JVM sans JAVA_HOME valide.")
    elif num_jars_found == 0 or not beliefdynamics_jar_found_in_glob:
        print(f"❌ Impossible de démarrer la JVM: classpath vide ({num_jars_found} JARs) ou JAR beliefdynamics non trouvé par glob.")
    else:
        try:
            print("\n⏳ Démarrage de la JVM...")
            jvm_args = [
                "-ea",
                f"-Djava.class.path={classpath}"
            ]
            if 'NATIVE_LIBS_DIR' in globals() and NATIVE_LIBS_DIR.exists() and any(NATIVE_LIBS_DIR.iterdir()):
                 native_path_arg = f"-Djava.library.path={NATIVE_LIBS_DIR.resolve()}"
                 jvm_args.append(native_path_arg)
                 print(f"   Argument JVM ajouté : {native_path_arg}")

            jpype.startJVM(*jvm_args, convertStrings=False)
            jvm_started_in_this_cell = True

            print("   Enregistrement des domaines JPype (org, java, net)...")
            jpype.imports.registerDomain("org")
            jpype.imports.registerDomain("java")
            jpype.imports.registerDomain("net")
            print("✅ JVM démarrée et domaines enregistrés.")

        except Exception as e:
            print(f"\n❌❌❌ Erreur LORS du démarrage de la JVM : {e} ❌❌❌")
            if hasattr(e, 'stacktrace'): print("\n--- Stacktrace Java ---\n" + e.stacktrace() + "\n-----------------------")
else:
    print("ℹ️ JVM déjà en route.")
    try:
        from org.tweetyproject.logics.pl.syntax import Proposition
    except ImportError:
        try:
            print("   Ré-enregistrement des domaines JPype (la JVM persistait peut-être)...")
            jpype.imports.registerDomain("org"); jpype.imports.registerDomain("java"); jpype.imports.registerDomain("net")
            print("   Domaines ré-enregistrés.")
        except Exception as e_re_reg:
             print(f"   ⚠️ Erreur lors de la ré-enregistrement des domaines : {e_re_reg}")


# --- Test d'import post-démarrage/vérification ---
print("\nEffectuant des tests d'import Java plus spécifiques...")
imports_ok = True
missing_imports = []
if jpype.isJVMStarted():
    try:
        print("   Test imports critiques...")
        info_obj_loaded = False
        try:
            from org.tweetyproject.beliefdynamics import InformationObject
            print("   ✔️ Import direct InformationObject réussi.")
            info_obj_loaded = True
        except ImportError:
            print("   ⚠️ Import direct InformationObject échoué. Tentative JClass...")
            try:
                 InformationObject = jpype.JClass("org.tweetyproject.beliefdynamics.InformationObject")
                 print("   ✔️ Chargement InformationObject via JClass réussi.")
                 info_obj_loaded = True
            except Exception as e_jclass_info:
                 print(f"   ❌ Chargement InformationObject via JClass échoué: {e_jclass_info}")
                 missing_imports.append("beliefdynamics.InformationObject")
                 imports_ok = False
        except Exception as e_other_info:
             print(f"   ❌ Erreur inattendue lors de l'import/chargement de InformationObject: {e_other_info}")
             missing_imports.append("beliefdynamics.InformationObject")
             imports_ok = False

        print("   Test autres imports nécessaires...")
        try: from org.tweetyproject.commons import Formula; print("   ✔️ Import commons.Formula OK.")
        except ImportError: missing_imports.append("commons.Formula"); imports_ok = False
        try: from org.tweetyproject.logics.pl.syntax import Proposition; print("   ✔️ Import logics.pl.syntax.* OK.")
        except ImportError: missing_imports.append("logics.pl.syntax.*"); imports_ok = False
        try: from org.tweetyproject.arg.dung.syntax import Argument; print("   ✔️ Import arg.dung.syntax.* OK.")
        except ImportError: missing_imports.append("arg.dung.syntax.*"); imports_ok = False
        try: from java.util import ArrayList; print("   ✔️ Import java.util.* OK.")
        except ImportError: missing_imports.append("java.util.*"); imports_ok = False

    except Exception as e_other:
            print(f"❌ Erreur inattendue pendant test import : {e_other}")
            imports_ok = False

    # Categorize failures: critical vs optional
    critical_failures = [m for m in missing_imports if "InformationObject" not in m]
    optional_failures = [m for m in missing_imports if "InformationObject" in m]
    
    if critical_failures:
        print(f"\n‼️ Des imports Java CRITIQUES ont echoue : {', '.join(critical_failures)}")
        print("   Cela indique un probleme de classpath ou des JARs manquants/corrompus.")
        print("   Verifiez les messages d'erreur ci-dessus et la presence des JARs dans 'libs/'.")
        if not beliefdynamics_jar_found_in_glob and "beliefdynamics" in ",".join(critical_failures):
             print("   **Le JAR 'beliefdynamics' n'a pas ete trouve par le script, ce qui explique l'echec.**")
        print("   La suite du notebook ne fonctionnera probablement pas.")
    elif optional_failures:
        print(f"\nℹ️ Certains imports optionnels ont echoue : {', '.join(optional_failures)}")
        print("   Note: InformationObject n'existe pas dans Tweety 1.28 (refactoring API).")
        print("   Seule la section Revision de Croyances (CrMas) sera affectee.")
        print("   Le reste du notebook (PL, FOL, DL, Argumentation) fonctionne normalement.")
    else:
        print("\n✔️ Tests d'imports de base reussis.")
else:
    print("\n⚠️ Tests d'imports sautés car la JVM n'a pas démarré ou a échoué au démarrage.")


--- 1.6 Démarrage de la JVM via JPype ---

‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️
‼️ SI DES JARs ONT ÉTÉ TÉLÉCHARGÉS (Cellule 7), REDÉMARREZ LE NOYAU MAINTENANT ‼️
‼️ Kernel -> Restart Kernel...                                         ‼️
‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️‼️
🔍 Recherche JDK portable dans l'arborescence projet...
  Scan du répertoire: jdk-17-portable
✅ JDK portable trouvé: d:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\Tweety\jdk-17-portable\zulu17.50.19-ca-jdk17.0.11-win_x64
🏠 Utilisation du JDK portable: d:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\Tweety\jdk-17-portable\zulu17.50.19-ca-jdk17.0.11-win_x64

Classpath (35 JARs trouvé(s)) assemblé.

--- DEBUG CLASSPATH ---
Longueur totale: 3897 caractères
✔️ JAR 'org.tweetyproject.beliefdynamics' TROUVÉ dans la liste: ...mbolicAI\Tweety\libs\org.tweetyproject.beliefdynamics-1.29-with-dependencies.jar
--- FIN DEBUG CLASSPATH ---

ℹ️ JVM déjà en route.

Effectuant d

#### Test Approfondi de Compatibilité API

La cellule suivante effectue un test spécifique pour la classe `InformationObject` du module `beliefdynamics`.

**Contexte** : Cette classe a été modifiée dans Tweety 1.28 lors d'un refactoring de l'API. Le test ci-dessous permet de déterminer si votre version de Tweety supporte les fonctionnalités de révision de croyances multi-agents (CrMas).

**Résultat attendu** :
- **Tweety 1.27 et antérieur** : Import réussi
- **Tweety 1.28+** : Import échoue (comportement normal, seule la section CrMas est affectée)

In [19]:
# --- Cellule 13.1 : Test Minimal Import InformationObject ---
print("\n--- Test Minimal Import InformationObject ---")
minimal_import_ok = False
info_obj_class_ref = None

if 'jpype' in globals() and jpype.isJVMStarted():
    try:
        # Essayer import direct
        from org.tweetyproject.beliefdynamics import InformationObject
        print("✔️ Import direct org.tweetyproject.beliefdynamics.InformationObject RÉUSSI.")
        info_obj_class_ref = InformationObject # Garder la référence
        minimal_import_ok = True
    except ImportError as e_imp:
        print(f"⚠️ Import direct échoué: {e_imp}. Tentative JClass...")
        try:
            info_obj_class_ref = jpype.JClass("org.tweetyproject.beliefdynamics.InformationObject")
            print("✔️ Chargement via JClass org.tweetyproject.beliefdynamics.InformationObject RÉUSSI.")
            minimal_import_ok = True
        except Exception as e_jclass:
            print(f"❌ Chargement via JClass ÉCHOUÉ: {e_jclass}")
    except Exception as e_other:
        print(f"❌ Erreur inattendue lors du test d'import minimal: {e_other}")

    if minimal_import_ok:
         # Test d'instanciation simple (nécessite d'autres classes, mais juste pour voir)
         try:
             from org.tweetyproject.logics.pl.syntax import Proposition
             from org.tweetyproject.agents import DummyAgent
             p = Proposition("test_prop")
             ag = DummyAgent("test_agent")
             # Assumons que info_obj_class_ref est la classe chargée
             # Note: Nécessite de caster les arguments pour le constructeur Java
             test_info_obj = info_obj_class_ref(jpype.JObject(p, jpype.JClass("org.tweetyproject.logics.pl.syntax.PlFormula")),
                                                jpype.JObject(ag, jpype.JClass("org.tweetyproject.agents.Agent")))
             print(f"✔️ Instanciation test de InformationObject réussie: {test_info_obj}")
         except Exception as e_inst:
             print(f"⚠️ L'import/chargement a réussi, MAIS l'instanciation test a échoué: {e_inst}")
             print("   (Cela peut être dû à des dépendances manquantes pour le constructeur)")

else:
    print("ℹ️ Test sauté (JVM non démarrée).")

# Mettre à jour la variable globale pour la cellule CrMas
# Écraser la valeur précédente si elle existait
CrMas_Imports_OK = minimal_import_ok
print(f"\n==> Résultat du test minimal: CrMas_Imports_OK = {CrMas_Imports_OK}")


--- Test Minimal Import InformationObject ---
⚠️ Import direct échoué: Failed to import 'org.tweetyproject.beliefdynamics.InformationObject'. Tentative JClass...
❌ Chargement via JClass ÉCHOUÉ: Class org.tweetyproject.beliefdynamics.InformationObject is not found

==> Résultat du test minimal: CrMas_Imports_OK = False


### 1.7 Concepts Clés de Tweety et JPype
<a id="1.7"></a>

Avant de plonger dans les exemples, comprenons quelques concepts fondamentaux de Tweety et comment nous interagissons avec eux via JPype :

**Concepts Tweety :**

*   **Signature** (`org.tweetyproject.logics.[logic].syntax.[Logic]Signature`) : Définit le vocabulaire d'une logique.
*   **Formula** (`org.tweetyproject.logics.[logic].syntax.[Logic]Formula`) : Représente une formule bien formée.
*   **BeliefBase** (`org.tweetyproject.logics.[logic].syntax.[Logic]BeliefSet`) : Un ensemble de formules (base de connaissances).
*   **Interpretation** (`org.tweetyproject.logics.[logic].semantics.Interpretation`) : Assigne une signification sémantique (ex: `PossibleWorld`).
*   **Parser** (`org.tweetyproject.logics.[logic].parser.[Logic]Parser`) : Analyse chaînes/fichiers vers `Formula`/`BeliefSet`.
*   **Reasoner** (`org.tweetyproject.logics.[logic].reasoner.[Logic]Reasoner`) : Implémente le raisonnement (`query`, `getModels`). Des raisonneurs par défaut peuvent être définis.
*   **Argumentation Frameworks** (`org.tweetyproject.arg.*`): Classes pour les cadres (`DungTheory`, `AspicArgumentationTheory`, etc.) et composants (`Argument`, `Attack`, `Support`).

**Interaction Python-Java avec JPype :**

*   **Imports**: Grâce à `jpype.imports.registerDomain("org", alias="org")` (exécuté dans la cellule de démarrage JVM), on peut importer les classes Java comme des modules Python :
    ```python
    from org.tweetyproject.logics.pl.syntax import Proposition
    from org.tweetyproject.arg.dung.syntax import Argument, Attack
    from java.util import ArrayList # Également possible via registerDomain("java", alias="java")
    ```
*   **Instanciation**: `p = Proposition("a")`.
*   **Appel de Méthodes**: `kb.add(p)`.
*   **Types Primitifs**: Conversion auto (int, float, bool, str).
*   **Collections Java**: Utiliser les types Java explicites (`ArrayList`, `HashSet` depuis `java.util`).
*   **Surcharge (Overloading)**: Si ambiguïté, caster avec `JObject(variable, ClasseJava)` ou `JInt()`, `JString()`, etc. (depuis `jpype.types`).
*   **Exceptions Java**: Attraper avec `except jpype.JException as e_java:`. Accéder au message avec `e_java.message()` et à la trace avec `e_java.stacktrace()`.

---

## Notebooks de la série Tweety

Après avoir exécuté ce notebook de configuration, vous pouvez explorer:

| # | Notebook | Thème |
|---|----------|-------|
| 2 | [Tweety-2-Basic-Logics](Tweety-2-Basic-Logics.ipynb) | Logique Propositionnelle et FOL |
| 3 | [Tweety-3-Advanced-Logics](Tweety-3-Advanced-Logics.ipynb) | DL, Modale, QBF, Conditional |
| 4 | [Tweety-4-Belief-Revision](Tweety-4-Belief-Revision.ipynb) | Révision de croyances, MUS, MaxSAT |
| 5 | [Tweety-5-Abstract-Argumentation](Tweety-5-Abstract-Argumentation.ipynb) | Dung, CF2, Génération |
| 6 | [Tweety-6-Structured-Argumentation](Tweety-6-Structured-Argumentation.ipynb) | ASPIC+, DeLP, ABA, ASP |
| 7 | [Tweety-7-Advanced-Argumentation](Tweety-7-Advanced-Argumentation.ipynb) | (Original complet - référence) |
| 7a | [Tweety-7a-Extended-Frameworks](Tweety-7a-Extended-Frameworks.ipynb) | ADF, Bipolar, WAF, SAF, SetAF, Extended |
| 7b | [Tweety-7b-Ranking-Probabilistic](Tweety-7b-Ranking-Probabilistic.ipynb) | Ranking Semantics, Probabiliste |
| 8 | [Tweety-8-Agent-Dialogues](Tweety-8-Agent-Dialogues.ipynb) | Agents, Dialogues argumentatifs |
| 9 | [Tweety-9-Preferences](Tweety-9-Preferences.ipynb) | Préférences, Théorie du vote |

> **Note**: Le notebook original Tweety-7 est conservé pour référence. La version splitée (7a + 7b) couvre le même contenu de manière plus détaillée.

---

**Navigation**: Index | [Suivant: Tweety-2-Basic-Logics →](Tweety-2-Basic-Logics.ipynb)